In [4]:
import pandas as pd

ruta_dataset = "./movies_dataset.csv"
df = pd.read_csv(ruta_dataset)

C:\Users\User\AppData\Local\Temp\ipykernel_13296\3517079211.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_dataset)


In [5]:
print(df.head())

                               belongs_to_collection      budget  \
0  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000.0   
1                                                NaN  65000000.0   
2  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0.0   
3                                                NaN  16000000.0   
4  {'id': 96871, 'name': 'Father of the Bride Col...         0.0   

                                              genres     id original_language  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862                en   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844                en   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602                en   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357                en   
4                     [{'id': 35, 'name': 'Comedy'}]  11862                en   

                                            overview popularity  \
0  Led by Woody, Andy's toys live hap

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   belongs_to_collection     4491 non-null   object 
 1   budget                    45379 non-null  float64
 2   genres                    45379 non-null  object 
 3   id                        45379 non-null  object 
 4   original_language         45368 non-null  object 
 5   overview                  44438 non-null  object 
 6   popularity                45377 non-null  object 
 7   production_companies      45379 non-null  object 
 8   production_countries      45379 non-null  object 
 9   release_date              45376 non-null  object 
 10  revenue                   45379 non-null  float64
 11  runtime                   45130 non-null  float64
 12  spoken_languages          45376 non-null  object 
 13  status                    45296 non-null  object 
 14  taglin

In [14]:
print(df.shape)

(45466, 18)


In [9]:
#eliminacion columnas solicitadas
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df = df.drop(columns=columnas_a_eliminar)

In [9]:
print(df.columns)

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'collection_id',
       'collection_name'],
      dtype='object')


In [23]:
#rellenar con 0 en las columnas solicitadas
df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [24]:
print(df[['revenue', 'budget']].isnull().sum())

revenue    0
budget     0
dtype: int64


In [16]:
#eliminacion de valores nulos de la columna solicitada
df = df.dropna(subset=['release_date'])

In [17]:
print(df['release_date'].isnull().sum())

0


In [22]:
# Convertir las columnas 'revenue' y 'budget' a numérico (reemplazando valores no válidos por NaN)
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [25]:
#creacion de la columna solicitada
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] 
                        if row['budget'] > 0 else 0, axis=1)

In [26]:
# Verificar los resultados
print(df[['revenue', 'budget', 'return']].head())

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


In [28]:
# Asegurarse de que la columna 'release_date' esté en formato de fecha (AAAA-mm-dd)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Eliminar valores nulos en 'release_date' (fechas no válidas)
#df = df.dropna(subset=['release_date'])

# Crear la columna 'release_year' extrayendo el año de la fecha de estreno
df['release_year'] = df['release_date'].dt.year

# Verificar los resultados
print(df[['release_date', 'release_year']].head())

  release_date  release_year
0   1995-10-30        1995.0
1   1995-12-15        1995.0
2   1995-12-22        1995.0
3   1995-12-22        1995.0
4   1995-02-10        1995.0


In [8]:
#desanidar columna belongs_to_collection
import ast
def extract_collection_info(row):
    
    if pd.isna(row):
        return None, None
    try:
        collection = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(collection, dict):
            return collection.get('id'), collection.get('name')
    except (ValueError, SyntaxError):
        return None, None

# Aplicar la función
df[['collection_id', 'collection_name']] = df['belongs_to_collection'].apply(
    extract_collection_info).apply(pd.Series)


In [11]:
#desanidar columna production_companies
def extract_company_names(row):
    if pd.isna(row):
        return None
    try:
        companies = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(companies, list):
            return [company.get('name') for company in companies if isinstance(company, dict) and 'name' in company]
    except (ValueError, SyntaxError):
        return None

# Aplicar la función
df['production_company_names'] = df['production_companies'].apply(extract_company_names)


In [12]:
#desanidar columna genres
def extract_genre_names(row):
    if pd.isna(row):
        return None
    try:
        genres = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(genres, list):
            return [genre.get('name') for genre in genres if isinstance(genre, dict) and 'name' in genre]
    except (ValueError, SyntaxError):
        return None

# Aplicar la función a la columna 'genres'
df['genre_names'] = df['genres'].apply(extract_genre_names)


In [14]:
# Guardar el DataFrame en un archivo CSV
df.to_csv("movies_dataset_final.csv", index=False)

print("Archivo CSV guardado exitosamente como 'movies_dataset_final.csv'.")

Archivo CSV guardado exitosamente como 'movies_dataset_final.csv'.
